In [1]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

2025-07-01 09:56:14.475991: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751363774.902075      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751363774.999060      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Load dữ liệu ảnh
def load_data(data_dir, img_size=224):
    X, y = [], []
    for label, folder in enumerate(['with_mask', 'without_mask']):
        path = os.path.join(data_dir, folder)
        for img_name in os.listdir(path):
            img_path = os.path.join(path, img_name)
            img = cv2.imread(img_path)
            if img is None:
                continue
            img = cv2.resize(img, (img_size, img_size))
            X.append(img)
            y.append(label)
    return np.array(X), np.array(y)

# Load data
X, y = load_data('/kaggle/input/face-mask-dataset/data', img_size=224)
X = X / 255.0  # chuẩn hóa
y = np.array(y)

# Chia train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

# Tăng cường dữ liệu
datagen = ImageDataGenerator(rotation_range=15, zoom_range=0.2, horizontal_flip=True)
datagen.fit(X_train)

In [3]:
# Xây dựng mô hình CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification
])



/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1751363876.105586      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1751363876.106265      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [4]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Huấn luyện
model.fit(datagen.flow(X_train, y_train, batch_size=32), 
          validation_data=(X_test, y_test), 
          epochs=10)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


I0000 00:00:1751363972.748435      98 service.cc:148] XLA service 0x7a7a38004740 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1751363972.749884      98 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1751363972.749912      98 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1751363973.185268      98 cuda_dnn.cc:529] Loaded cuDNN version 90300


  1/189 ━━━━━━━━━━━━━━━━━━━━ 28:39 9s/step - accuracy: 0.5625 - loss: 0.6977

I0000 00:00:1751363979.287054      98 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


189/189 ━━━━━━━━━━━━━━━━━━━━ 73s 341ms/step - accuracy: 0.7651 - loss: 0.5466 - val_accuracy: 0.8915 - val_loss: 0.2470
Epoch 2/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 57s 300ms/step - accuracy: 0.8890 - loss: 0.2626 - val_accuracy: 0.9007 - val_loss: 0.2327
Epoch 3/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 57s 303ms/step - accuracy: 0.9059 - loss: 0.2263 - val_accuracy: 0.8915 - val_loss: 0.2942
Epoch 4/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 58s 308ms/step - accuracy: 0.9077 - loss: 0.2340 - val_accuracy: 0.9193 - val_loss: 0.1989
Epoch 5/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 56s 298ms/step - accuracy: 0.9223 - loss: 0.1903 - val_accuracy: 0.9252 - val_loss: 0.1803
Epoch 6/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 56s 296ms/step - accuracy: 0.9284 - loss: 0.1870 - val_accuracy: 0.9305 - val_loss: 0.1820
Epoch 7/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 57s 302ms/step - accuracy: 0.9341 - loss: 0.1646 - val_accuracy: 0.9398 - val_loss: 0.1667
Epoch 8/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 56s 295ms/step - accuracy: 0.9418 - loss: 0.1476 - val

In [5]:
model.save('mask_detector.h5')


In [6]:
model.evaluate(X_test, y_test)

48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.9505 - loss: 0.1534


[0.14581769704818726, 0.9516876339912415]